<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Time Series Analysis  - Recurrent Neural Network & LSTM - Weather Australia
</b></font> </br></p>

---

# 0  | Install & Import
***

In [ ]:
# Install

In [ ]:
# Import
from pandas import read_csv, DataFrame, concat
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from keras.utils import set_random_seed, plot_model

from tensorflow import keras
from tensorflow.config.experimental import enable_op_determinism
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from keras.models import Sequential, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import plotly.express as px

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1 | Understand
---

<p><font color='black' size="5">
Anwendungsfall
</font></p>

---   


Dieser Datensatz enthält ungefähr 10 Jahre Wetterbeobachtungen von Hobart in Australien.

Die `Mittlere Temperatur` ist die vorherzusagende Zielvariable.
Prognostizieren die `Mittlere Temperatur`, indem Sie das Modelle mit der Zielvariablen trainieren.


[Info](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package)

[DataSet](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package)





In [ ]:
data = read_csv(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/weather_hobart.csv"
)

<p><font color='black' size="5">
EDA (Exploratory Data Analysis)   
</font></p>

<p><font color='black' size="4">
Visualisierung der Daten
</font></p>

In [ ]:
fig = px.line(
    data, x="YearMonth", y="MedTemp", title="Weather Hobart", width=1000, height=500
)
fig.show()

# 2 | Prepare
---


<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Duplikate ermitteln/löschen</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>
✅ Train-Test-Split durchführen</br>

<p><font color='black' size="5">
Übergreifende Parameter
</font></p

In [ ]:
past_steps = 12
future_steps = 1
num_features = 1


<p><font color='black' size="5">
Train-Test-Split
</font></p>

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data.MedTemp, data.MedTemp, test_size=0.2, shuffle=False
)
data_train.shape, data_test.shape, target_train.shape, target_test.shape

In [ ]:
fig = px.line(
    data, x="YearMonth", y="MedTemp", title="Weather Hobart", width=1000, height=500
)
fig.add_vline(
    x=data.YearMonth[target_train.shape[0]],
    line_width=3,
    line_dash="dash",
    line_color="red",
)
fig.show()


<p><font color='black' size="5">
Aufbau der Zeitscheiben
</font></p>

In [ ]:
generator = TimeseriesGenerator(
    target_train.values, target_train.values, length=past_steps, batch_size=future_steps
)

# # Beispiel erste Zeitscheibe
x, y = generator[0]
print(x.shape, y.shape)
print(f"data: {list(x)}")
print("target:", y)

# 3 | Modeling
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

<p><font color='black' size="5">
Zufallszahl initialisieren
</font></p>

In [ ]:
set_random_seed(42)
enable_op_determinism()

<p><font color='black' size="5">
Modellaufbau
</font></p>

[Core layers](https://keras.io/api/layers/core_layers/)    
[Recurrent layers](https://keras.io/api/layers/recurrent_layers/)   
[Layer activation functions](https://keras.io/api/layers/activations/)   


In [ ]:
model = Sequential(name="Zeitreihenanalyse")
model.add(Input(shape=(past_steps, num_features)))
model.add(LSTM(256, activation="relu", return_sequences=True))
model.add(LSTM(128, activation="relu", return_sequences=True))
model.add(LSTM(64, activation="relu", return_sequences=True))
model.add(LSTM(32, activation="relu", return_sequences=False))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
# Visualisierung neuronales Netz
plot_model(
    model,
    to_file="nn_structure.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    dpi=100,
    expand_nested=True,
    show_layer_activations=True
)

In [ ]:
# Anzahl Parameter je Layer
for layer in model.layers:
    print(f"{layer.name}: {layer.count_params()} Parameter")


<p><font color='black' size="5">
Compile
</font></p>

In [ ]:
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

<p><font color='black' size="5">
Callbacks
</font></p>

In [ ]:
early = EarlyStopping(monitor="mae", patience=2)

reduce_lr = ReduceLROnPlateau(
    monitor="loss",
    factor=0.1,
    patience=2,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)

check = ModelCheckpoint(filepath="model.keras", monitor="mae", save_best_only=True)

<p><font color='black' size="5">
Training
</font></p>

In [ ]:
model.fit(generator, epochs=20, batch_size=256, callbacks=[early, check])

In [ ]:
save_history = model.history.history

<p><font color='black' size="5">
Loss-Entwickung
</font></p>

In [ ]:
title_ = "Loss-Entwicklung"
px.line(
    y=save_history["loss"],
    title=title_,
    labels={"x": "Epochen", "y": "Loss-Wert"},
    width=800,
    height=400,
)

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
Rollierende Prognose
</font></p>

In [ ]:
### Bestes Modell laden
model = load_model("model.keras")

In [ ]:
# Rollierende Vorhersage - Logik für eine "echte" Prognose in die Zukunft
initial_sequence_generator = TimeseriesGenerator(
    target_train.tail(past_steps + 1).values,
    target_train.tail(past_steps + 1).values,
    length=past_steps,
    batch_size=future_steps,
)

# x, y = initial_sequence_generator[0]
# print(x.shape, y.shape)

# Leere Liste für die Vorhersagen
target_test_pred = []
# Initialisierung für 1. Prognose
initial_sequence, y_dummy = initial_sequence_generator[0]

# Anzahl der Schritte, die Sie in die Zukunft vorhersagen möchten

for _ in range(target_test.shape[0]):
    # Machen Sie eine Vorhersage mit der aktuellen Sequenz
    current_prediction = model.predict(initial_sequence)

    # Fügen Sie die Vorhersage zu Ihrer Vorhersageliste hinzu
    target_test_pred.append(current_prediction[0, 0])

    # Aktualisieren Sie die Sequenz: Entfernen Sie den ältesten Punkt und fügen Sie die neue Vorhersage hinzu
    initial_sequence = np.roll(initial_sequence, -1, axis=1)
    initial_sequence[0, -1] = current_prediction

In [ ]:
cube = data[-len(target_test) :]
cube["Real"] = DataFrame(target_test)
cube["Pred"] = target_test_pred

<p><font color='black' size="5">
Bestimmtheitsmass
</font></p>

In [ ]:
f"Bestimmtheitsmass: {r2_score(cube['Real'].values, cube['Pred'].values):.3f}"

<p><font color='black' size="5">
Vergleich Zeitreihen
</font></p>

In [ ]:
fig = px.line(
    cube,
    x="YearMonth",
    y=["Real", "Pred"],
    title="Zeitreihenanalyse - Real vs. Predict",
    width=1000,
    height=500,
)
fig.show()

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>